# Summary

Test out new batch query functionality.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import openai
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend, query_kwargs_grid, MOCKS, postprocess_gpt_response, \
    containerize, truncate_at_first_stop, query_gpt_mock
from jabberwocky.utils import strip
from htools import *

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [4]:
gpt = GPTBackend()
gpt.switch('repeat')

Switching openai backend to "repeat".


In [5]:
def postprocess_response(response, n, trunc_full=True, strip_output=True, 
                         **kwargs):
    text, full_response = containerize(*response)
    print('TEXT:', text)
    print('FULL:', full_response)

    # Manually check for stop phrases because most backends either don't
    # or truncate AFTER the stop phrase which is rarely what we want.
    stop = kwargs.get('stop', [])
    clean_text = []
    clean_full = []
    for i, (text_, resp_) in enumerate(zip(text, full_response)):
        text_ = truncate_at_first_stop(
            text_,
            stop_phrases=stop,
            finish_reason=resp_.get('finish_reason', ''),
            trunc_full=trunc_full,
            trunc_partial=True
        )
        clean_text.append(strip(text_, strip_output))
        clean_full.append({**resp_, 'prompt_index': i // n})

    return clean_text, clean_full

In [6]:
def test_backend(backend, results=None):
    def gen():
        yield from results.kwargs
        
    with gpt(backend):
        kwargs_list = []
        resp_list = []
        gen_kwargs = gen if results else query_kwargs_grid
        for i, kwargs in enumerate(gen_kwargs(), start=1):
            print(f'\n\n{i}.')
            if isinstance(kwargs['prompt'], str):
                np = 1
            else:
                np = len(kwargs['prompt'])
            expected_prompt_idx = list(range(np))
                
            # Get completion, either from new query or cached result.
            if results:
                res = results.res[i - 1]
            else:
                res = gpt.query(**kwargs)
                
            # Print results.
            if kwargs['stream']:
                cur = []
                actual_prompt_idx = []
                for tok, tok_full in res:
                    cur.append((tok, tok_full))
                    print(tok)
                    print('\t' + str(tok_full) + '\n')
                    if tok_full['finish_reason']: print('\n---\n')
                    actual_prompt_idx.append(tok_full.get('prompt_index', -1))
            else:
                texts, fulls = res
                print('TEXTS:', texts)
                print('FULLS:', fulls)
                actual_prompt_idx = [full['prompt_index'] for full in fulls]
                cur = res
            
            actual_prompt_idx = sorted(set(actual_prompt_idx))
            assert actual_prompt_idx == expected_prompt_idx, \
                    f'Expected prompt indices {expected_prompt_idx}, ' \
                    f'got {actual_prompt_idx}'
            print(spacer())

            kwargs_list.append(kwargs)
            resp_list.append(cur)
    return Results(kwargs=kwargs_list, res=resp_list)

Issues

- no prompt_index in stream=True mode for either repeat or banana (pretty sure not for paid backends either)

In [7]:
repeat_res = test_backend('repeat')

Switching openai backend to "repeat".
np>1: False
nc>1: False
stream: False


1.
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
TEXTS: ['YESTERDAY WAS']
FULLS: [{'prompt_index': 0}]

-------------------------------------------------------------------------------

np>1: False
nc>1: False
stream: True


2.
{'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
YESTERDAY 
	{'index': 0, 'prompt_index': 0, 'finish_reason': None}

WAS 
	{'index': 0, 'prompt_index': 0, 'finish_reason': 'dummy'}


---


-------------------------------------------------------------------------------

np>1: False
nc>1: True
stream: False


3.
{'n': 2, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'repeat', 'que

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:375: UserWarning: Unused kwargs {'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3} received by query_gpt_repeat.
  warnings.warn(f'Unused kwargs {kwargs} received by query_gpt_repeat.')
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:674: UserWarning: strip_output=True is not supported in stream mode. Automatically setting it to False.
  warnings.warn('strip_output=True is not supported in stream '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:679: UserWarning: Streaming mode does not support manual truncation of stop phrases and your current backend has limited support for truncation.
  'Streaming mode does not support manual truncation of '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:375: UserWarning: Unused kwargs {'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3} received by query_gpt_repeat.
  warnings.warn(f'Unused kwargs {kwargs} received by qu

In [37]:
# Even though it's free, try to avoid hitting the API too much just in case 
# they have some rate limit.
# banana_res = test_backend('banana')
# save(banana_res, 'data/tmp/banana_res.pkl')
# banana_res = load('data/tmp/banana_res.pkl')

In [38]:
banana_res = test_backend('banana', results=banana_res)

Switching openai backend to "banana".


1.
TEXTS: ['the first time']
FULLS: [{'id': '73a2b095-3bd2-438b-9bc9-078aa4003226', 'message': 'success', 'created': 1649906615, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' the first time', 'input': 'Yesterday was'}], 'prompt_index': 0}]

-------------------------------------------------------------------------------



2.
 
	{'id': '9478d077-a6e9-4b0a-8a12-3b7f971f8050', 'message': 'success', 'created': 1649906615, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a weird day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index': 0, 'finish_reason': None}

a 
	{'id': '9478d077-a6e9-4b0a-8a12-3b7f971f8050', 'message': 'success', 'created': 1649906615, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a weird day', 'input': 'Yesterday was'}], 'index': 0, 'prompt_index': 0, 'finish_reason': None}

weird 
	{'id': '9478d077-a6e9-4b0a-8a12-3b7f971f8050', 'message': 'success', 'created': 1649906615, 'apiVersion': '26

In [41]:
mock_res = test_backend('mock')

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.
Switching openai backend to "mock".
np>1: False
nc>1: False
stream: False


1.
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock'}}
TEXTS: ['a typical rainy']
FULLS: [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x1236e62b0> JSON: {
  "text_offset": [
    0,
    2,
    10
  ],
  "token_logprobs": [
    -1.3818359375,
    -6.06640625,
    -5.671875
  ],
  "tokens": [
    " a",
    " typical",
    " rainy"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    },
    {
      " big": -1.83984375,
      " day": -2.53125,
      " very": -2.916015625
    },

 a
	{'finish_reason': None, 'index': 0, 'logprobs': <OpenAIObject at 0x123602048> JSON: {
  "text_offset": [
    0
  ],
  "token_logprobs": [
    -1.3818359375
  ],
  "tokens": [
    " a"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    }
  ]
}, 'text': ' a', 'token_index': 0, 'prompt_index': 0}

 day
	{'finish_reason': None, 'index': 0, 'logprobs': <OpenAIObject at 0x123602200> JSON: {
  "text_offset": [
    2
  ],
  "token_logprobs": [
    -2.53125
  ],
  "tokens": [
    " day"
  ],
  "top_logprobs": [
    {
      " big": -1.83984375,
      " day": -2.53125,
      " very": -2.916015625
    }
  ]
}, 'text': ' day', 'token_index': 1, 'prompt_index': 0}

 when
	{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x1236023b8> JSON: {
  "text_offset": [
    6
  ],
  "token_logprobs": [
    -2.763671875
  ],
  "tokens": [
    " when"
  ],
  "top_logprobs": [
    {
      " for": -2.296875,
      " of": -1.1

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:404: UserWarning: query_gpt_mock received unused kwargs: {'engine_i': 0, 'max_tokens': 3, 'logprobs': 3}
  warnings.warn(f'query_gpt_mock received unused kwargs: {kwargs}')
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:674: UserWarning: strip_output=True is not supported in stream mode. Automatically setting it to False.
  warnings.warn('strip_output=True is not supported in stream '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:679: UserWarning: Streaming mode does not support manual truncation of stop phrases and your current backend has limited support for truncation.
  'Streaming mode does not support manual truncation of '


In [44]:
hf_res = test_backend('huggingface')

Switching openai backend to "huggingface".
np>1: False
nc>1: False
stream: False


1.
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'huggingface', 'query_func': 'query_gpt_huggingface'}}
TEXTS: ['the day that']
FULLS: [{'generated_text': ' the day that', 'prompt_index': 0}]

-------------------------------------------------------------------------------

np>1: False
nc>1: False
stream: True


2.
{'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'huggingface', 'query_func': 'query_gpt_huggingface'}}
 
	{'generated_text': ' the day that', 'index': 0, 'prompt_index': 0, 'finish_reason': None}

the 
	{'generated_text': ' the day that', 'index': 0, 'prompt_index': 0, 'finish_reason': None}

day 
	{'generated_text': ' the day that', 'index': 0, 'prompt_index': 0, 'finish_reason': None}

that 
	{'generated_text': ' the day that', 'index': 0, 

MockFunctionException: 400 Client Error: Bad Request for url: https://api-inference.huggingface.co/models/EleutherAI/gpt-neo-125M

## Scratch

In [51]:
# 2 prompts, 1 completion per prompt, stream=False
p2c1s0 = MOCKS[True, False, False]
p1c2s0 = MOCKS[False, True, False]

In [52]:
texts, fulls = containerize(*postprocess_gpt_response(p2c1s0))
postprocess_response((texts, fulls), n=1)

(['24 years ago', 'months did it'],
 [{'finish_reason': 'length',
   'index': 0,
   'logprobs': <OpenAIObject at 0x125bacaf0> JSON: {
     "text_offset": [
       0,
       3,
       9
     ],
     "token_logprobs": [
       -8.1484375,
       -1.931640625,
       -0.8271484375
     ],
     "tokens": [
       " 24",
       " years",
       " ago"
     ],
     "top_logprobs": [
       {
         " a": -1.3818359375,
         " my": -2.384765625,
         " the": -1.8720703125
       },
       {
         " hours": -1.21484375,
         " years": -1.931640625,
         "th": -1.8212890625
       },
       {
         " ago": -0.8271484375,
         " since": -1.0576171875,
         " to": -2.794921875
       }
     ]
   },
   'text': ' 24 years ago',
   'token_index': 0,
   'prompt_index': 0},
  {'finish_reason': 'length',
   'index': 1,
   'logprobs': <OpenAIObject at 0x125bacca8> JSON: {
     "text_offset": [
       0,
       7,
       11
     ],
     "token_logprobs": [
       -5.464843

In [73]:
texts, fulls = postprocess_gpt_response(p1c2s0)
postprocess_response((texts, fulls), n=2)

TEXT: [' my mother�', ' the Academy Awards']
FULL: [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x1245f1990> JSON: {
  "text_offset": [
    0,
    3,
    10
  ],
  "token_logprobs": [
    -2.384765625,
    -5.4375,
    -0.44677734375
  ],
  "tokens": [
    " my",
    " mother",
    "\ufffd"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    },
    {
      " birthday": -2.107421875,
      " first": -1.3818359375,
      " last": -2.130859375
    },
    {
      "'s": -1.509765625,
      "-": -2.630859375,
      "\ufffd": -0.44677734375
    }
  ]
}, 'text': ' my mother�', 'token_index': 0}, {'finish_reason': 'length', 'index': 1, 'logprobs': <OpenAIObject at 0x1245f1b48> JSON: {
  "text_offset": [
    0,
    4,
    12
  ],
  "token_logprobs": [
    -1.8720703125,
    -8.796875,
    -0.1270751953125
  ],
  "tokens": [
    " the",
    " Academy",
    " Awards"
  ],
  "top_logprobs": [
    {
      " a":

(['my mother�', 'the Academy Awards'],
 [{'finish_reason': 'length',
   'index': 0,
   'logprobs': <OpenAIObject at 0x1245f1990> JSON: {
     "text_offset": [
       0,
       3,
       10
     ],
     "token_logprobs": [
       -2.384765625,
       -5.4375,
       -0.44677734375
     ],
     "tokens": [
       " my",
       " mother",
       "\ufffd"
     ],
     "top_logprobs": [
       {
         " a": -1.3818359375,
         " my": -2.384765625,
         " the": -1.8720703125
       },
       {
         " birthday": -2.107421875,
         " first": -1.3818359375,
         " last": -2.130859375
       },
       {
         "'s": -1.509765625,
         "-": -2.630859375,
         "\ufffd": -0.44677734375
       }
     ]
   },
   'text': ' my mother�',
   'token_index': 0,
   'prompt_index': 0},
  {'finish_reason': 'length',
   'index': 1,
   'logprobs': <OpenAIObject at 0x1245f1b48> JSON: {
     "text_offset": [
       0,
       4,
       12
     ],
     "token_logprobs": [
       -1.

In [74]:
texts

[' my mother�', ' the Academy Awards']

In [115]:
with gpt('mock'):
    res = gpt.query('ac', n=1, stream=False)

Switching openai backend to "mock".
{'n': 1, 'stream': False, 'prompt': 'ac', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock'}}
>>> response: ([' a typical rainy'], [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x125a06e08> JSON: {
  "text_offset": [
    0,
    2,
    10
  ],
  "token_logprobs": [
    -1.3818359375,
    -6.06640625,
    -5.671875
  ],
  "tokens": [
    " a",
    " typical",
    " rainy"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    },
    {
      " big": -1.83984375,
      " day": -2.53125,
      " very": -2.916015625
    },
    {
      " Monday": -2.974609375,
      " day": -1.095703125,
      " work": -2.94140625
    }
  ]
}, 'text': ' a typical rainy', 'token_index': 0}])
TEXT: [' a typical rainy']
FULL: [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x125a06e08> JSON: {
  "text_offset": [
    0,
    2,
    10
  ],
  "token_logprobs":

In [116]:
res

(['a typical rainy'],
 [{'finish_reason': 'length',
   'index': 0,
   'logprobs': <OpenAIObject at 0x125a06e08> JSON: {
     "text_offset": [
       0,
       2,
       10
     ],
     "token_logprobs": [
       -1.3818359375,
       -6.06640625,
       -5.671875
     ],
     "tokens": [
       " a",
       " typical",
       " rainy"
     ],
     "top_logprobs": [
       {
         " a": -1.3818359375,
         " my": -2.384765625,
         " the": -1.8720703125
       },
       {
         " big": -1.83984375,
         " day": -2.53125,
         " very": -2.916015625
       },
       {
         " Monday": -2.974609375,
         " day": -1.095703125,
         " work": -2.94140625
       }
     ]
   },
   'text': ' a typical rainy',
   'token_index': 0,
   'prompt_index': 0}])